# 豆瓣电影的多方法解析
https://movie.douban.com/tag/Top100

### 1. 分析网页，确认爬取目标的数据类型。
   - #### 打开 [目标url](https://movie.douban.com/tag/Top100), 定位数据位置
   ![爬取源网址](./images/index.png)  
   - #### 定位需要的数据位置，查看爬取目标。  
   ![爬取目标](./images/data_target.png)  
   由图可得，我们需要的数据分别为，['海报', '电影名', '上映日期', '演员', '评分', '评价人数']
   - #### 查看请求，分析数据来源请求（F12 >> network 打开请求界面，如下图）
   ![抓包准备](./images/capture_package_ready.png)  
   - #### 确认数据请求来源(Ctrl + F 搜索: 辛德勒)
   ![抓包准备](./images/confirm_request.png)  
   上图可知，该请求只有一个，所以就能轻松的确定来源拉！ 
   - #### 查看headers，分析请求报文
   ![请求分析](./images/requests_ann.png)  
   分析结果如图，所以我们可以得出以下结论：  
   
    
| 信息     | 结果                                  |
| -------- | -------------------------------------|
| 请求地址 | <https://movie.douban.com/tag/Top100> |
| 请求方法 | Get                                   |
| 响应格式 | text 文本                             |
| 编码     | UTF-8                                |

## 2. 利用requests进行请求测试
`requests.get`  
定义请求函数，`get_data`  
返回`text`数据  

In [1]:
import requests
from requests.exceptions import HTTPError

In [2]:
def get_data(url):
    
    response = requests.get(url)
    if response.status_code == requests.codes.ok:  # 检测状态码
        return response.text
    else:
        response.raise_for_status()  # 4xx 5xx 时,引出错误 代替 raise requests.exception.HTTPError

In [3]:
url = "https://movie.douban.com/tag/Top100"

In [4]:
data = get_data(url)  # 获取数据

## 3. 提取数据
   - 正则提取
   - BeautifulSoup 提取
   - Xpath 提取
   - scrapy 混合提取

### 1. 正则提取
- 观察数据位置
![电影名，海报地址](./images/re_name_poster.png)

In [5]:
import re

In [15]:
# 设置提取表达式 用 .*? 去掉变化值 (.*?) 保留需要的变化值 re.S 表示 . 可以匹配换行等字符
poster_pattern = re.compile(r"""<a class="nbg" href="https://movie.douban.com/subject/\d+/" title=".*?">.*?<img src="(.*?)" width="75" alt=".*?" class="">.*?</a>""", re.S)  # 海报的正则表达式
movie_name_pattern = re.compile(r"""<a href=".*?" class="">(.*?)<span style="font-size:13px;">(.*?)</span>.*?</a>""", re.S)  # 电影名正则表达式
poster_res = re.findall(poster_pattern, data)
movie_name_res = re.findall(movie_name_pattern, data)
poster_res, movie_name_res

([],
 [('\n        <tr class="item">\n            <td width="100" valign="top">\n                \n\n                    <a class="nbg" href="https://movie.douban.com/subject/1295124/"  title="辛德勒的名单">\n                        <img src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p492406163.jpg" width="75" alt="辛德勒的名单" class=""/>\n                    </a>\n            </td>\n\n            <td valign="top">\n                \n\n                <div class="pl2">\n\n                    <a href="https://movie.douban.com/subject/1295124/"  class="">\n                        辛德勒的名单\n                        / ',
   '舒特拉的名单(港) / 辛德勒名单'),
  ('\n        <tr class="item">\n            <td width="100" valign="top">\n                \n\n                    <a class="nbg" href="https://movie.douban.com/subject/1292262/"  title="美国往事">\n                        <img src="https://img1.doubanio.com/view/photo/s_ratio_poster/public/p477229647.jpg" width="75" alt="美国往事" class=""/>\n        